In [1]:

from gurobipy import Model, GRB, quicksum
import pandas as pd
from code_map import meters, markets
import numpy as np
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime
import pytz
import openpyxl
from itertools import combinations, product


/Users/sandermeland/Documents/Jobb/Volte/master-kode/code_map/markets.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffr_df["FFR-Flex Price [EUR/MW]"][(pd.Timestamp(year = year, month =10, day = 30, hour = 0, tz = "Europe/Oslo") < ffr_df["Time(Local)"]) &
/Users/sandermeland/Documents/Jobb/Volte/master-kode/code_map/markets.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffr_df["FFR-Profil Price [EUR/MW]"][(pd.Timestamp(year = year, month = 9, day = 3, hour = 0, tz = "Europe/Oslo") < ffr_df["Time(Local)"]) &
/Users/sandermeland/Documents/Jobb/Volte/master-kode/code_map/markets.py:84: SettingWithCopyWarning: 

In [2]:
market_list = markets.all_market_list
market_dict = {market.name : market for market in market_list}
#timeframe = pd.date_range(start=pd.Timestamp(year= markets.year, month= markets.start_month, day = markets.start_day, hour = markets.start_hour), 
                             # end= pd.Timestamp(year = markets.year, month = markets.end_month, day = markets.end_day, hour = markets.end_hour), freq="H", tz = "Europe/Oslo")

hour = pd.Timestamp(year= markets.year, month= markets.start_month, day = markets.start_day, hour = markets.start_hour, tz = "Europe/Oslo")
market_names = [market.name for market in market_list]
power_meter_dict = meters.power_meters


In [3]:
#H = timeframe # set of hours in timehorizone
M = market_list # set of markets
L = list(power_meter_dict.values()) # set of meters

#load_pr_hour = [L] * len(H) # set of meters per hour

In [4]:
# For multiple hours
"""L_v = np.array([[meter.flex_volume["value"].loc[meter.flex_volume["Time(Local)"] == hour].values[0] for meter in L] for hour in H]) # set of flex volumes for meters

L_s = np.array([[meter.sleep_time for meter in L]] * len(H)) # set of sleep times for meters
L_r = np.array([[meter.response_time for meter in L]] * len(H)) # set of response times for meters

M_p = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of prices for markets
M_v = np.array([[market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of volumes for markets

M_m = np.array([[market.min_volume *0.1 for market in M]] * len(H)) # set of min values for markets
M_r = np.array([[market.response_time for market in M]] * len(H)) # set of response times for markets
M_s = np.array([[market.sleep_time for market in M]] * len(H)) # set of sleep times for markets"""


'L_v = np.array([[meter.flex_volume["value"].loc[meter.flex_volume["Time(Local)"] == hour].values[0] for meter in L] for hour in H]) # set of flex volumes for meters\n\nL_s = np.array([[meter.sleep_time for meter in L]] * len(H)) # set of sleep times for meters\nL_r = np.array([[meter.response_time for meter in L]] * len(H)) # set of response times for meters\n\nM_p = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of prices for markets\nM_v = np.array([[market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of volumes for markets\n\nM_m = np.array([[market.min_volume *0.1 for market in M]] * len(H)) # set of min values for markets\nM_r = np.array([[market.response_time for market in M]] * len(H)) # set of response times for markets\nM_s = np.array([[market.sleep_time for market in M]] * len(H)) # set of sleep times for markets'

In [5]:
L_v = [meter.flex_volume["value"].loc[meter.flex_volume["Time(Local)"] == hour].values[0] for meter in L]  # set of flex volumes for meters

#L_s = [meter.sleep_time * 0.1 for meter in L] # set of sleep times for meters
#L_r = [meter.response_time * 0.1 for meter in L] # set of response times for meters

M_p = [market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in M] # set of prices for markets
M_v = [market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in M] # set of volumes for markets

M_m = [market.min_volume for market in M] # set of min values for markets
#M_r = [market.response_time  for market in M] # set of response times for markets
#M_s = [market.sleep_time  for market in M] # set of sleep times for markets

In [19]:
L_v

[0.000552,
 0.0068784000000000015,
 2.6400000000000005e-05,
 2.9999999999999997e-05,
 0.0014712,
 0.0,
 0.00244,
 0.0022800000000000003,
 0.0013000000000000002,
 0.0026984,
 0.01576,
 0.0064,
 0.0048720000000000005,
 0.0016,
 0.0097,
 3.2000000000000005e-05,
 0.021540000000000004,
 0.0074800000000000005,
 0.001176,
 0.003488,
 0.0001,
 0.0015840000000000001,
 0.0048000000000000004,
 0.001376,
 0.002736,
 0.004272000000000001,
 0.003,
 0.0005600000000000001,
 0.0005920000000000001,
 0.002352,
 0.0004160000000000001,
 0.000144,
 0.0,
 0.0006900000000000001,
 0.0026888000000000003,
 0.005184000000000001,
 0.00222,
 0.021680000000000005,
 0.000984,
 0.0037600000000000003,
 0.002112,
 0.00023999999999999998,
 0.0026700000000000005,
 0.0005099999999999999,
 0.00252,
 0.000784,
 0.0006900000000000001,
 0.004200000000000001,
 0.0054,
 0.000928,
 0.0012640000000000001,
 0.0019199999999999998,
 0.0,
 0.01088,
 0.013519999999999999,
 0.005340000000000001,
 0.05168,
 0.00108,
 0.001950000000000000

In [6]:
im = np.zeros((len(M), len(L)))
for m in range(len(M)):
    for l in range(len(L)):
        im[m,l] = M_p[m] * L_v[l] 

In [7]:
im[1,1]

0.08769960000000003

# Optimization model for the simplified model

In [8]:
simple_model = Model(name="Simple Optimization Model")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-18


# Decision variables

In [9]:
# Binary variables
x = simple_model.addVars(len(M), len(L), vtype = GRB.BINARY, name = "x") # load l is bid to market m at hour h

y = simple_model.addVars(len(M), vtype = GRB.BINARY, name = "y") # market m is in use at hour h

# Continuous variables
#d = simple_model.addVars(len(M) * len(H), vtype = GRB.CONTINUOUS, name = "d") # flex volume bid in to market m in hour h


# Constraints

In [10]:
"""# one market per load per hour
simple_model.addConstrs((sum(x[m,l,h] for m in range(len(M))) <= 1 for l in range(len(L)) for h in range(len(H))), name = "one_market_per_load_per_hour")
# minimum volume for up markets
simple_model.addConstrs((sum(L_v[h,l] * x[m,l,h] for l in range(len(L)))  >= M_m[h,m] for m in range(len(M)) for h in range(len(H))), name = "min_bid_vol_up")
# minimum volume for down markets

# response time faster than market criteria
simple_model.addConstrs((np.mean([L_r[h,l]* x[m,l, h] for l in range(len(L))]) <= M_r[h,m] for m in range(len(M)) for h in range(len(H))), name = "response_time")
# sleep time shorter than market criteria
simple_model.addConstrs((np.mean([L_s[h,l]* x[m,l,h] for l in range(len(L))]) <= M_s[h,m] for m in range(len(M)) for h in range(len(H))), name = "sleep_time")
"""

'# one market per load per hour\nsimple_model.addConstrs((sum(x[m,l,h] for m in range(len(M))) <= 1 for l in range(len(L)) for h in range(len(H))), name = "one_market_per_load_per_hour")\n# minimum volume for up markets\nsimple_model.addConstrs((sum(L_v[h,l] * x[m,l,h] for l in range(len(L)))  >= M_m[h,m] for m in range(len(M)) for h in range(len(H))), name = "min_bid_vol_up")\n# minimum volume for down markets\n\n# response time faster than market criteria\nsimple_model.addConstrs((np.mean([L_r[h,l]* x[m,l, h] for l in range(len(L))]) <= M_r[h,m] for m in range(len(M)) for h in range(len(H))), name = "response_time")\n# sleep time shorter than market criteria\nsimple_model.addConstrs((np.mean([L_s[h,l]* x[m,l,h] for l in range(len(L))]) <= M_s[h,m] for m in range(len(M)) for h in range(len(H))), name = "sleep_time")\n'

In [11]:
M_m

[5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10]

In [12]:
sum(L_v[l] for l in range(len(L))) - M_m[8]

-8.000372800000001

In [13]:
# Only one hour
simple_model.addConstrs((sum(x[m,l] for m in range(len(M))) <= 1 for l in range(len(L))), name = "one_market_per_load") # this works

#simple_model.addConstrs(y[m] = 1 if quicksum(x[m, l] for l in range(len(L))) for m in range(len(M)) >= 0 else 0 for m in range(len(M))), name = "market_in_use"

simple_model.addConstrs(y[m] == quicksum(x[m, l] for l in range(len(L))) for m in range(len(M)))

# minimum volume for up markets
#simple_model.addConstrs(((quicksum(L_v[l] for l in range(len(L))) - M_m[m]) * x[m,l]  >= 0 for m in range(len(M))), name = "min_bid_vol_up")

#simple_model.addConstrs((quicksum(L_v[l] * x[m, l] for l in range(len(L))) >= M_m[m] for m in range(len(M))), name="min_bid_vol_up")

simple_model.addConstrs(((quicksum(L_v[l] * x[m, l] for l in range(len(L))) >= M_m[m] * y[m]) for m in range(len(M))), name="min_bid_vol_up")


# minimum volume for down markets

# response time faster than market criteria
#simple_model.addConstrs((np.mean([L_r[l]* x[m,l] for l in range(len(L))]) <= M_r[m] for m in range(len(M))), name = "response_time")
# sleep time shorter than market criteria
#simple_model.addConstrs((np.mean([L_s[l]* x[m,l] for l in range(len(L))]) <= M_s[m] for m in range(len(M))), name = "sleep_time")


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>}

In [14]:
#obj_fn = sum(M_p[h,m] * L_v[h,l] * x[m,l,h] for m in range(len(M)) for l in range(len(L)) for h in range(len(H)))
obj_fn = quicksum(im[m,l] * x[m, l] for m in range(len(M)) for l in range(len(L)))

simple_model.setObjective(obj_fn, GRB.MAXIMIZE)

In [15]:
#simple_model.setParam('OutputFlag', False)
simple_model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 355 rows, 4592 columns and 13566 nonzeros
Model fingerprint: 0xdd3f04e1
Variable types: 0 continuous, 4592 integer (4592 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+01]
  Objective range  [9e-05, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 355 rows and 4592 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


In [18]:
counter = 0
for v in simple_model.getVars():
    counter += 1
    print(v.varName, " : ", v.x)
    

x[0,0]  :  0.0
x[0,1]  :  0.0
x[0,2]  :  0.0
x[0,3]  :  0.0
x[0,4]  :  0.0
x[0,5]  :  0.0
x[0,6]  :  0.0
x[0,7]  :  0.0
x[0,8]  :  0.0
x[0,9]  :  0.0
x[0,10]  :  0.0
x[0,11]  :  0.0
x[0,12]  :  0.0
x[0,13]  :  0.0
x[0,14]  :  0.0
x[0,15]  :  0.0
x[0,16]  :  0.0
x[0,17]  :  0.0
x[0,18]  :  0.0
x[0,19]  :  0.0
x[0,20]  :  0.0
x[0,21]  :  0.0
x[0,22]  :  0.0
x[0,23]  :  0.0
x[0,24]  :  0.0
x[0,25]  :  0.0
x[0,26]  :  0.0
x[0,27]  :  0.0
x[0,28]  :  0.0
x[0,29]  :  0.0
x[0,30]  :  0.0
x[0,31]  :  0.0
x[0,32]  :  0.0
x[0,33]  :  0.0
x[0,34]  :  0.0
x[0,35]  :  0.0
x[0,36]  :  0.0
x[0,37]  :  0.0
x[0,38]  :  0.0
x[0,39]  :  0.0
x[0,40]  :  0.0
x[0,41]  :  0.0
x[0,42]  :  0.0
x[0,43]  :  0.0
x[0,44]  :  0.0
x[0,45]  :  0.0
x[0,46]  :  0.0
x[0,47]  :  0.0
x[0,48]  :  0.0
x[0,49]  :  0.0
x[0,50]  :  0.0
x[0,51]  :  0.0
x[0,52]  :  0.0
x[0,53]  :  0.0
x[0,54]  :  0.0
x[0,55]  :  0.0
x[0,56]  :  0.0
x[0,57]  :  0.0
x[0,58]  :  0.0
x[0,59]  :  0.0
x[0,60]  :  0.0
x[0,61]  :  0.0
x[0,62]  :  0.0
x[

In [ ]:
counter

In [ ]:
print(simple_model.printStats())

In [ ]:
print(simple_model.display())